In [ ]:
%matplotlib inline
import matplotlib.pyplot as plt
import numpy as np
from tqdm import trange, tqdm_notebook
import tensorflow as tf

In [ ]:
num_points = 5000
my_data = []
for i in range(num_points):
    x1 = np.random.normal(0, 1)
    y1 = np.sin(x1) + np.random.normal(0, 0.1)
    my_data.append([x1,y1])

x_data = [ v[0] for v in my_data ]
y_data = [ v[1] for v in my_data ]
plt.plot(x_data, y_data, 'ro')
plt.show()

In [ ]:
print(np.shape(x_data))
print(np.ndim(x_data))

In [ ]:
_x_data = tf.reshape(x_data, [len(x_data), 1])
print(_x_data)

In [ ]:
__x_data = tf.expand_dims(x_data, 1)
print(__x_data)

In [ ]:
x_data = np.expand_dims(x_data, 1)
y_data = np.expand_dims(y_data, 1)

In [ ]:
input_data = tf.placeholder(tf.float32, shape=[None,1])
output_data = tf.placeholder(tf.float32, shape=[None,1])

In [ ]:
W1 = tf.Variable(tf.random_uniform([1, 5], -1.0, 1.0))
W2 = tf.Variable(tf.random_uniform([5, 3], -1.0, 1.0))
W_out = tf.Variable(tf.random_uniform([3, 1], -1.0, 1.0))

In [ ]:
hidden1 = tf.nn.sigmoid(tf.matmul(input_data, W1))
hidden2 = tf.nn.sigmoid(tf.matmul(hidden1, W2))
output = tf.matmul(hidden2, W_out)

In [ ]:
loss = tf.reduce_mean(tf.square(output - output_data))

In [ ]:
global_step = tf.Variable(0, trainable=False)

In [ ]:
optimizer = tf.train.AdamOptimizer(0.5)
train = optimizer.minimize(loss, global_step=global_step)

In [ ]:
sess = tf.Session()
sess.run(tf.global_variables_initializer())

In [ ]:
writer = tf.summary.FileWriter('./mini_graph', sess.graph)
tf.summary.scalar('loss', loss)
tf.summary.histogram('histogram_loss', loss)
summary_op = tf.summary.merge_all()

In [ ]:
def shuffle_data(x_train, y_train):
    temp_index = np.arange(len(x_train))
    np.random.shuffle(temp_index)
    x_temp = np.zeros(x_train.shape)
    y_temp = np.zeros(y_train.shape)
    x_temp = x_train[temp_index]
    y_temp = y_train[temp_index]
    return x_temp, y_temp

In [ ]:
%%time
BATCH_SIZE = 1000
BATCH_NUM = int(len(x_data)/BATCH_SIZE)

#for step in range(5000):
for step in tqdm_notebook(range(5000), desc='1st loop'):
    index = 0
    x_data, y_data = shuffle_data(x_data, y_data)
    for batch_iter in range(BATCH_NUM-1):
    #for batch_iter in tqdm_notebook(range(BATCH_NUM-1), desc='2nd loop', leave=False):
        feed_dict = {input_data:x_data[index:index+BATCH_SIZE], output_data:y_data[index:index+BATCH_SIZE]}
        _, summary = sess.run([train, summary_op], feed_dict=feed_dict)
        writer.add_summary(summary, global_step=index)
        index += BATCH_SIZE

In [ ]:
plt.plot(x_data, y_data, 'ro')
plt.plot(x_data, sess.run(output, feed_dict={input_data:x_data}), 'bo')
plt.xlabel('x')
plt.xlim(-4, 3)
plt.ylabel('y')
plt.ylim(-1.5, 1.5)
plt.show()

In [ ]:
sess.close()

In [ ]:
## 텐서보드 실행방법
## tensorboard --logdir=로그가_저장된_폴더
## 주의 : tensorboard --logdir='mini_graph' <-- 처럼 따옴표로 묶으면 안됨! 에러!( 예: 'mini_graph')
## tensorboard --logdir=mini_graph

## 브라우저로 http://localhost:6006 으로 접속!

In [ ]:
# end